In [ ]:
# Importing files
from nupack import * # imports nupack
import pandas as pd # imports pandas
import numpy as np
import matplotlib.pyplot as plt # imports plotting functions
import re
import random
import csv
import ast 
import sys

In [ ]:
######################################## CHANGE ########################################
# Additional Strands
BindingSite = "CAATAAGCGATGCGCCCTCGCCTGGGGGCCTAGTCCTCT" # 5'-3'
sBinding = "GTTATTCGCTACG" #3'-5' # short binding site
Segment2 = "CCTATGCGTGCTACCGTGAA" #5'-3' # ye duan's primer
yeDuanAptamer = "AGCAGCACAGAGGTCAGATGCAATAAGCGATGCGCCCTCGCCTGGGGGCCTAGTCCTCTCCTATGCGTGCTACCGTGAA" #5'-3'

# Primers
# Mark 10 
fwdPrimer = "CGTTCAATGATATAGGCTCGCATAGGAGACAC" #5'-3'
cFwdPrimer = "GCAAGTTACTATATCCGAGCGTATCCTCTGTG" #3'-5' #"CACAGAGGATACGCTCGGATATAGTAACTTGC"# Reverse complement #5'-3'
rvsPrimer = "CAAAGCTAGAAGAGCAGCACAGAGGTCAGATG" #5'-3' 
cRvsPrimer = "GTTTCGATCTTCTCGTCGTGTCTCCAGTCTAC" #3'-5' #"CATCTGACCTCTGTGCTGCTCTTCTAGCTTTG"# Reverse complement 5'-3'

# Constants
nitrogenousBase = ("A","T","C","G")
nucleicAcid = 'DNA'
temperature = 37                                                                    # In celcius. Can change to kelvin in my_model or simple conversion.
ionNa = 0.100                                                                       # Nupack boundaries molar [0.05, 1.1] molar
ionMg = 0.002                                                                       # Nupack boundaries molar [0.0, 0.2] molar
concentration = 5e-7                                                                # In molar. Ex: 5e-7 is 0.5 uM
sizeMax = 2                                                                         # max size for complexes
gcMax = 0.70                                                                        # GC content max value
gcMin = 0.30                                                                        # GC content min value
probeLenMax = 46                                                                    # Max length of PROBE binding site
probeLenMin = 46                                                                    # Min length of PROBE binding site
gapLenMax = 20                                                                      # Max length of between PROBE and PRIMER binding site
gapLenMin = 20                                                                      # Min length of between PROBE and PRIMER binding site
n = 20000                                                                           # total number of aptamer and blocker pairs
minGibbs = -12                                                                      # minimum accepted Gibbs Free Energy in kcal/mol
maxGibbs = -5                                                                       # maximum accepted Gibbs Free Energy kcal/mol
gPattern = re.compile(r'^#.{0.2}(GGG)')                                             # No consecutive G bases from the first 0 to 2 bases
consecutiveBP = re.compile(r'(GGGGGG|CCCCC|AAAAA|TTTTT)')                           # Max mononucleotide 5 (ex: GGGGG, etc) [Note: G is maxed to 6 since pre-existing aptamer has 5 already]

#  CHANGE NAMES EVERYTIME! IT WILL OVERWRITE FILES WITH THE SAME NAME
openExistingFile_Boolean = False                                                    # Boolean to access sequences from a different file
openExistingFile_Name = 'Mark9.csv'                                                 # Name of existing csv file
saveProbeGap_Sequences = True                                                       # Boolean to save the probe and gap sequences generated in one run
saveSequences_Name = "Mark2Version2.csv"                                            # Name of existing csv file
saveData_PreFilters_Name = "Analysized Aptamer and Blockers Mark2Version2.xlsx"     # Name of excel file containing prefilter analysis on sequences 
saveData_PostFilters_Name = "1_20Gap_46Probe_M2V2.xlsx"                             # Name of excel file containing final sequences after filters

# Physical model
my_model = Model(material=nucleicAcid, celsius=temperature, sodium=ionNa, magnesium=ionMg) 
###########################################################################################

In [ ]:
#Functions
def gcContents(seq):
    ''' gcContents finds gc fraction of the sequence 
    Calculates the total G's and C's in the string
    Parameters:     seq                 string of oligonucleotides 
    Returns:        counter/len(seq)    decimal fraction
    '''
    counter = seq.count("G")+seq.count("C")
    if len(seq) > 0:
        gC = counter/len(seq)
    return gC

def cDNAorRNA(seq):
    ''' cDNAorRNA returns the complementary sequence of input
    Parameters:     seq                 Oligonucleotide sequence
    Returns:        seq                 Complementary oligonucleotide sequence
    '''
    relationship = {'A':'T','T':'A','C':'G','G':'C'}
    return "".join([relationship[base] for base in seq])

def reverseComp(seq):
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    return ''.join(complement[base] for base in reversed(seq))

def randomSeq(base, maxLen, minLen):
    ''' randSeq creates a random oligonucleotide sequence
    Takes the parameter 'bases' and randomizes it
    Parameters:     bases               bases of nucleotides
                    max                 Maximum sequence length
                    min                 Minimum sequence length
    Returns:        seq                 Oligonucleotide sequence
    '''
    counter = range(random.randint(minLen,maxLen))
    for i in counter:
        seq = "".join([random.choice(base) for i in counter]) 
    return seq

def generateSequences (bases, minGC, maxGC, minLen, maxLen, trigger, limit):
    ''' generateSequences generates x amount of random oligonucleotide sequences
    that satisfys user's GC content and sequence length range 
    Parameters:     bases               bases of nucleotides
                    minGC               Minimum GC content
                    maxGC               Maximum GC content
                    maxLen              Maximum sequence length
                    minLen              Minimum sequence length
                    trigger             Determines where to store data
                    limit               Number of sequences to generate
    '''
    global cProbe, probeLen, gap, gapLen
    for i in range(limit):
        gC = 0
        while gC > maxGC or gC < minGC:
            sequence = randomSeq(bases, maxLen, minLen)
            gC = gcContents(sequence)
        if trigger == "Probe":
            cProbe.append(sequence)
            probeLen.append(len(sequence))
        elif trigger == "Gap":
            gap.append(sequence)
            gapLen.append(len(sequence))

    if trigger == "Probe":
        print(f"Completed generating {len(cProbe)} probe region sequences")
    elif trigger == "Gap":
        print(f"Completed generating {len(gap)} gap region sequences")

def packagingData(data, seq, strandList, gEnergyList, mfeList):
    ''' packagingData sorts data to specific lists 
    Parameters:     data                Unfiltered (raw) data 
                    seq                 Oligonucleotide sequence
                    strandList          Lists to store seq
                    gEnergyList         Lists to store complex gibbs energy
                    mfeList             Lists to store minimum free energy
    '''
    # data: model, strand name, pFunc, gibbs, mfe, none, pairs
    if seq != None:
        strandList.append(seq)
    gEnergyList.append(data[3])
    mfeList.append(next((energy for structure, energy, stackE in data[8]), None))

def read_mark_csv(csv_filename):
    ''' read_mark_csv reads data saved into a csv file
    Parameters:     csv_filename        Name of csv file
    Returns:        gap_sequences       List of gap sequences
                    probe_sequences     List of probe sequences
    '''
    gap_sequences = []
    probe_sequences = []
    
    with open(csv_filename, 'rt') as csvFile:
        reader = csv.reader(csvFile)
        next(reader) 
        for row in reader:
            gap_sequences.append(row[1])  
            probe_sequences.append(row[2])
    return gap_sequences, probe_sequences

def graphingAptamerBlockerCAP(concList, index, titleName, labels, variable_1):
    ''' graphingConcentration3Strand graphs 3 strand data 
    Parameters:     concList            List of concentrations 
                    index               Sequence Pair index
                    titleName           Name of the title
                    labels              Labels for the legend
    '''
    concList = [np.array(conc) for conc in concList]
    a, aa, aaa, aab, aac, ab, abb, abc, ac, acb, acc,b, bb, bbb, bbc, bc, bcc, c, cc, ccc = concList
    total = [sum(complex) for complex in zip(*concList)]
    colors =    ["#1f77b4", #a blue
                "#ff7f0e", #b orange
                "#D7D731", #c yellow
                "#d62728", #ab red
                "#82011F", #abc rose
                "#FFBF07", #bc 
                "#2ca02c", #aa green
                "#206E20", #aaa dark green
                "#31BE94", #aab sea green
                "#9BEC04", #aac lime green
                "#5f8cbf",#abb light blue 
                "#98F5F9",#ac  ???
                "#A7B1BE", #acb grey
                "#DBAAE2", #acc pink??
                "#9467bd", #bb purple
                "#C58D85",#bbb coral
                "#b48ae6", #bbc lavendar
                "#3A3728", #bcc mud
                "#F4F4BD", #cc ray gold
                "#838307"#ccc dark gold
                ]
    plt.figure(figsize=(15,5))
    barWidth = 0.15
    xAxis = np.arange(len(index))
    offset1 = xAxis-barWidth
    offset2 = xAxis
    offset3 = xAxis+barWidth

    # Neutral
    plt.bar(offset2, a/total, barWidth, label = labels[0], color = colors[0])
    bar2 = a/total
    plt.bar(offset2, c/total, barWidth, label = labels[17], bottom = bar2, color = colors[2])
    bar2 += c/total
    
    # Benefitial 
    plt.bar(offset1, b/total, barWidth, label = labels[11], color = colors[1])
    bar1 = b/total
    plt.bar(offset1, ac/total, barWidth, label = labels[8], bottom = bar1, color = colors[11])
    bar1 += ac/total

    # Detrimental
    plt.bar(offset3, aa/total, barWidth, label = labels[1], color = colors[6])
    bar3 = aa/total
    plt.bar(offset3, aaa/total, barWidth, label = labels[2], bottom = bar3, color = colors[7])
    bar3 += aaa/total
    plt.bar(offset3, aab/total, barWidth, label = labels[3], bottom = bar3, color = colors[8])
    bar3 += aab/total
    plt.bar(offset3, aac/total, barWidth, label = labels[4], bottom = bar3, color = colors[9])
    bar3 += aac/total
    plt.bar(offset3, ab/total, barWidth, label = labels[5], bottom = bar3, color = colors[3])
    bar3 += ab/total
    plt.bar(offset3, abb/total, barWidth, label = labels[6], bottom = bar3, color = colors[10])
    bar3 += abb/total
    plt.bar(offset3, acb/total, barWidth, label = labels[9], bottom = bar3, color = colors[12])
    bar3 += acb/total
    plt.bar(offset3, acc/total, barWidth, label = labels[10], bottom = bar3, color = colors[13])
    bar3 += acc/total
    plt.bar(offset3, bb/total, barWidth, label = labels[12], bottom = bar3, color = colors[14])
    bar3 += bb/total
    plt.bar(offset3, bbb/total, barWidth, label = labels[13], bottom = bar3, color = colors[15])
    bar3 += bbb/total
    plt.bar(offset3, bbc/total, barWidth, label = labels[14], bottom = bar3, color = colors[16])
    bar3 += bbc/total
    plt.bar(offset3, bcc/total, barWidth, label = labels[16], bottom = bar3, color = colors[17])
    bar3 += bcc/total
    plt.bar(offset3, cc/total, barWidth, label = labels[18], bottom = bar3, color = colors[18])
    bar3 += cc/total
    plt.bar(offset3, ccc/total, barWidth, label = labels[19], bottom = bar3, color = colors[19])
    bar3 += ccc/total
    plt.bar(offset3, abc/total, barWidth, label = labels[7], bottom = bar3, color = colors[4])
    bar3 += abc/total
    plt.bar(offset3, bc/total, barWidth, label = labels[15], bottom = bar3, color = colors[5])
    bar3 += bc/total

    plt.legend(loc='right', bbox_to_anchor=(1.15,0.5))
    plt.title(f"{titleName}")
    if len(index) <= 300:
        plt.xticks(xAxis, index, rotation = -90)
    plt.ylabel('Probabilty')
    plt.xlabel('Sequence Index')
    plt.ylim(0,1)

    plt.figure(figsize=(5,5))
    barWidth = 0.15
    xAxis = np.arange(len(index))
    x1 = (ac/total)*100
    y1 = (b/total)*100
    
    unqiueLabels = {}
    shadingHue = ['orange', '#5f8cbf', 'Green']
    colorGroup = ['[0.5-0.65]', '(0.65-0.8]', '(0.8-1]'] 
    legendShapes = ['o','s','^'] #circle, square, triangle
    
    for i in range(len(variable_1)):
        val = variable_1[i]
        if val <= 0.65:
            num = 0
        elif 0.65 < val <= 0.8:
            num = 1
        else:
            num = 2
        
        plt.scatter(x1[i], y1[i], color=shadingHue[num], marker=legendShapes[num])
        plt.text(x1[i], y1[i] + 0.1, str(i+1), ha='center', va='bottom') #chr(65+i) =letters; str(i+1) =numbers
        
        if colorGroup[num] not in unqiueLabels:
            unqiueLabels[colorGroup[num]] = plt.Line2D([],[], color=shadingHue[num],
                                                       marker=legendShapes[num],
                                                       label=colorGroup[num],
                                                       linestyle = 'None')
           
    plt.xlabel('A.C (%)', fontsize = 14)
    plt.ylabel('B (%)', fontsize =14)
    plt.legend(
        handles=list(unqiueLabels.values()),
        loc='upper right',
        bbox_to_anchor=(1.35,1),
        title = 'Ratio of A.B in\nabsence of CAP')
    plt.title('Effect of CAP on Aptamer-Blocker System', fontsize=14)

def graphingConcentration3Strand(concList, index, titleName, labels):
    ''' graphingConcentration3Strand graphs 3 strand data 
    Parameters:     concList            List of concentrations 
                    index               Sequence Pair index
                    titleName           Name of the title
                    labels              Labels for the legend
    Returns:        total               List of the sums of concentrations present in their respective reaction
    '''
    concList = [np.array(conc) for conc in concList]
    a, aa, aaa, aab, aac, ab, abb, abc, ac, acb, acc,b, bb, bbb, bbc, bc, bcc, c, cc, ccc = concList
    total = [sum(complex) for complex in zip(*concList)]
    colors =    ["#E69F00", "#56B4E9", "#009E73", "#F0E442", "#0072B2", "#D55E00", "#CC79A7", "#999999",
                "#000000", "#FF0000", "#8E44AD", "#1F77B4", "#D62728", "#9467BD", "#2CA02C", "#FF7F0E",
                "#6B8E23", "#C71585", "#20B2AA", "#FFD700"]
    plt.figure(figsize=(15,5))
    barWidth = 0.15
    xAxis = np.arange(len(index))
    offset1 = xAxis-barWidth
    offset2 = xAxis
    offset3 = xAxis+barWidth

    # Neutral
    plt.bar(offset2, a/total, barWidth, label = labels[0], color = colors[0])
    bar2 = a/total
    plt.bar(offset2, b/total, barWidth, label = labels[11], bottom = bar2, color = colors[1])
    bar2 += b/total
    plt.bar(offset2, c/total, barWidth, label = labels[17], bottom = bar2, color = colors[2])
    bar2 += c/total
    
    # Benefitial 
    plt.bar(offset1, ab/total, barWidth, label = labels[5], color = colors[3])
    bar1 = ab/total
    plt.bar(offset1, abc/total, barWidth, label = labels[7], bottom = bar1, color = colors[4])
    bar1 += abc/total
    plt.bar(offset1, bc/total, barWidth, label = labels[15], bottom = bar1, color = colors[5])
    bar1 += bc/total

    # Detrimental
    plt.bar(offset3, aa/total, barWidth, label = labels[1], color = colors[6])
    bar3 = aa/total
    plt.bar(offset3, aaa/total, barWidth, label = labels[2], bottom = bar3, color = colors[7])
    bar3 += aaa/total
    plt.bar(offset3, aab/total, barWidth, label = labels[3], bottom = bar3, color = colors[8])
    bar3 += aab/total
    plt.bar(offset3, aac/total, barWidth, label = labels[4], bottom = bar3, color = colors[9])
    bar3 += aac/total
    plt.bar(offset3, abb/total, barWidth, label = labels[6], bottom = bar3, color = colors[10])
    bar3 += abb/total
    plt.bar(offset3, ac/total, barWidth, label = labels[8], bottom = bar3, color = colors[11])
    bar3 += ac/total
    plt.bar(offset3, acb/total, barWidth, label = labels[9], bottom = bar3, color = colors[12])
    bar3 += acb/total
    plt.bar(offset3, acc/total, barWidth, label = labels[10], bottom = bar3, color = colors[13])
    bar3 += acc/total
    plt.bar(offset3, bb/total, barWidth, label = labels[12], bottom = bar3, color = colors[14])
    bar3 += bb/total
    plt.bar(offset3, bbb/total, barWidth, label = labels[13], bottom = bar3, color = colors[15])
    bar3 += bbb/total
    plt.bar(offset3, bbc/total, barWidth, label = labels[14], bottom = bar3, color = colors[16])
    bar3 += bbc/total
    plt.bar(offset3, bcc/total, barWidth, label = labels[16], bottom = bar3, color = colors[17])
    bar3 += bcc/total
    plt.bar(offset3, cc/total, barWidth, label = labels[18], bottom = bar3, color = colors[18])
    bar3 += cc/total
    plt.bar(offset3, ccc/total, barWidth, label = labels[19], bottom = bar3, color = colors[19])
    bar3 += ccc/total

    plt.legend(loc='right', bbox_to_anchor=(1.2,0.5))
    plt.title(f"{titleName}")
    if len(index) <= 300:
        plt.xticks(xAxis, index, rotation = -90)
    plt.ylabel('Probabilty')
    plt.xlabel('Sequence Index')
    plt.ylim(0,1)
    return total

def rpaConcentrations(primerConcentration, blockerConcentration, blockers):
    ''' rpaConcentrations graphs 3 strand data particularly for RPA 
    Parameters:     primerConcentration     Concentration for the primers
                    blockerConcentration    Concentration for the blockers
                    blockers                List of blocker sequences (DNA)
    Returns:        concList                List of concentrations after reaction
    '''
    a, aa, aaa, aab, aac, ab, abb, abc, ac, acb, acc = [], [],[],[],[], [], [],[],[],[],[]
    b, bb, bbb, bbc, bc, bcc = [],[],[],[],[],[]
    c, cc, ccc, total = [], [], [], []
    concList = [a, aa, aaa, aab, aac, ab, abb, abc, ac, acb, acc,b, bb, bbb, bbc, bc, bcc, c, cc, ccc]

    for i in range(len(blockers)):
        A = Strand(fwdPrimer, material=nucleicAcid, name='A') # forward primer
        B = Strand(blockers[i], material=nucleicAcid, name='B') #Blocker
        C = Strand(rvsPrimer, material=nucleicAcid, name='C') # reverse primer
        
        # Specify complexes up to size 3
        t1 = Tube(strands={A: primerConcentration, B: blockerConcentration, C: primerConcentration},
                    complexes=SetSpec(max_size=3), name='t1')
        my_results = tube_analysis(tubes=[t1], model=my_model)
        complex_results = complex_analysis(complexes=t1,model=my_model, compute=['pfunc'])
        complex_results2 = complex_concentrations(tube=t1,data=complex_results)
        [concList[i].append(conc) for i, conc in enumerate(complex_results2.tubes[t1].complex_concentrations.values())]
        total.append(sum(complex_results2.tubes[t1].complex_concentrations.values()))
    return concList

In [ ]:
# Accessing previously saved sequences 
if openExistingFile_Boolean == True:
    csv.field_size_limit(sys.maxsize)

    # Lists
    index_list = []
    gap = []
    probe_sequences = []

    # CSV file
    with open(openExistingFile_Name, mode='r', newline='') as csv_file:
        csv_reader = csv.DictReader(csv_file)

        for row in csv_reader:
            index_list.append(row['Index'])
    
            forward_gap_sequence = ast.literal_eval(row['Forward Gap Sequence 3-5'])[0]
            gap.append(forward_gap_sequence)
            
            probe_sequence = ast.literal_eval(row['Probe Sequence 5-3'])
            probe_sequences.append(probe_sequence)

    probeLen, gapLen = [], []
    probeIndex = range(n)
    cProbe = probe_sequences[0]
    for probe in cProbe:
        probeLen.append(len(probe))

    for seq in gap:
        gapLen.append(len(seq))

In [ ]:
# Generating probe and gap sequences
# Main
if saveProbeGap_Sequences == True:
    cProbe, probeLen, probeIndex = [], [], []
    gap, gapLen = [], []
    generateSequences(nitrogenousBase, gcMin, gcMax, gapLenMin, gapLenMax, "Gap", 1)
    generateSequences(nitrogenousBase, gcMin, gcMax, probeLenMin, probeLenMax, "Probe", n)
    probeIndex = range(n)

    # Printing to csv file
    csvFile = open(saveSequences_Name, 'wt', newline='')
    writer = csv.writer(csvFile)    
    try:
        writer = csv.writer(csvFile)
        writer.writerow(('Index', 'Forward Gap Sequence 3-5', 'Probe Sequence 5-3'))
        for i in range(len(gap)):
            if i == 0:
                writer.writerow((i, gap, cProbe))
            else:
                writer.writerow((i, None, cProbe))
    finally:
        csvFile.close()

In [ ]:
# Processing Aptamer and Blocker strands
# Lists
strandA, strandAA, strandB, strandBB, strandAB = [], [], [], [], [] # strands
strandALen, strandBLen = [], []
gibbsEnergyA, gibbsEnergyAA, gibbsEnergyB, gibbsEnergyBB, gibbsEnergyAB = [], [], [], [], [] # gibbs free energy
mfeA, mfeAA, mfeB, mfeBB, mfeAB = [], [], [], [], [] # minimal free energy
concA, concB, concAA, concAB, concBB, totalConc = [], [], [], [], [], [] # concentration
concList = [concA, concAA, concAB, concB, concBB]

# Collecting Data
for i in range(len(probeIndex)):
    blockerSeq = cFwdPrimer+gap[0]+cDNAorRNA(cProbe[i])+cRvsPrimer+sBinding
    aptamerSeq = cProbe[i]+rvsPrimer+BindingSite+Segment2
    strandALen.append(len(aptamerSeq))
    strandBLen.append(len(blockerSeq))
    
    A = Strand(aptamerSeq, material=nucleicAcid, name = 'A') # aptamer sequence
    B = Strand(blockerSeq, material=nucleicAcid, name = 'B') # blocker sequence
    t1 = Tube(strands= {A: concentration, B: concentration}, complexes=SetSpec(max_size=sizeMax), name='t1')
    my_results = tube_analysis(tubes=[t1], model= my_model, compute=['mfe'])
    
    packagingData(my_results['(A)'], aptamerSeq, strandA, gibbsEnergyA, mfeA) # Collecting Strand A data
    packagingData(my_results['(A+A)'], None, strandAA, gibbsEnergyAA, mfeAA) # Collecting Strand A-A data
    packagingData(my_results['(B)'], blockerSeq, strandB, gibbsEnergyB, mfeB) # Collecting Strand B data
    packagingData(my_results['(B+B)'], None, strandBB, gibbsEnergyBB, mfeBB) # Collecting Strand B-B data
    packagingData(my_results['(A+B)'], None, strandAB, gibbsEnergyAB, mfeAB) # Collecting Strand A-B data

    complex_results = complex_analysis(complexes=t1,model=my_model, compute=['pfunc'])
    complex_results2 = complex_concentrations(tube=t1,data=complex_results)
    [concList[i].append(conc) for i, conc in enumerate(complex_results2.tubes[t1].complex_concentrations.values())]
    totalConc.append(sum(complex_results2.tubes[t1].complex_concentrations.values()))

    if i%250 == 0 and i != 0:
        print(i)

# Combining data for analysis
combinedData = np.column_stack((strandA, strandB,
                                gibbsEnergyA, gibbsEnergyB, gibbsEnergyAA, gibbsEnergyAB, gibbsEnergyBB,
                                mfeA, mfeB, mfeAA, mfeAB, mfeBB,
                                probeIndex, strandALen, strandBLen,
                                concA, concB, concAA, concAB, concBB, totalConc))

# Changing matrix to dataframe
labels = ['Aptamer (A)','Blocker (B)',
          'Gibbs Energy of A','Gibbs Energy of B', 'Gibbs Energy of AA','Gibbs Energy of AB','Gibbs Energy of BB',
          'MFE A', 'MFE B', 'MFE AA', 'MFE AB', 'MFE BB',
          'Index', 'Length of A', 'Length of B',
          'Concentration A', 'Concentration B', 'Concentration AA', 'Concentration AB', 'Concentration BB', 'Total Concentration']
labeledCombinedData = pd.DataFrame(combinedData, columns=labels)
labeledCombinedData.to_excel(saveData_PreFilters_Name)

In [ ]:
# Filtering by gibbs energy 
# List
filteredGibbsData = []

# Main
for i in range(len(probeIndex)):
    aptamerA = labeledCombinedData.iloc[i,0]
    blockers = labeledCombinedData.iloc[i,1]
    mfeAData = labeledCombinedData.iloc[i,7]
    mfeBData = labeledCombinedData.iloc[i,8]

    if gPattern.search(aptamerA[:5]) or gPattern.search(blockers[:5]):
        continue
    if consecutiveBP.search(aptamerA) or consecutiveBP.search(blockers):
        continue

    if minGibbs <= float(mfeAData) <= maxGibbs and minGibbs <= float(mfeBData) <= maxGibbs:
        rowData = labeledCombinedData.iloc[i].tolist()
        filteredGibbsData.append(rowData)

labeledFilteredGibbsData = pd.DataFrame(filteredGibbsData, columns=labeledCombinedData.columns)
filteredGibbsData = np.column_stack(labeledFilteredGibbsData.values.T)
print(f"Number of Sequence Pairs: {len(labeledFilteredGibbsData)}")

In [ ]:
# Filtering by Aptamer-Blocker concentration
# Main
dataAB = labeledFilteredGibbsData.iloc[:,18].astype(float)
dataTotal = labeledFilteredGibbsData.iloc[:,20].astype(float)
concFilter = (dataAB/dataTotal) > 0.5

filteredConcData = np.delete(labeledFilteredGibbsData.to_numpy(), np.where(~concFilter), axis=0)
labeledFilteredConcData = pd.DataFrame(filteredConcData, columns=labeledFilteredGibbsData.columns)
print(f"Number of Sequence Pairs: {len(labeledFilteredConcData)}")

In [ ]:
# Graphing and processing the Aptamer, Blocker, and CAP sequences
# Lists
a, aa, aaa, aab, aac, ab, abb, abc, ac, acb, acc = [], [],[],[],[], [], [],[],[],[],[]
b, bb, bbb, bbc, bc, bcc = [],[],[],[],[],[]
c, cc, ccc, totalABCapConc = [], [], [], []
concList = [a, aa, aaa, aab, aac, ab, abb, abc, ac, acb, acc,b, bb, bbb, bbc, bc, bcc, c, cc, ccc]

# Constants
capstoneData = filteredConcData
capstoneIndex = [row[12] for row in filteredConcData]
barWidth = 0.15

# Main
for i in range(len(capstoneData)):
    A = Strand(capstoneData[i,0], material=nucleicAcid, name='A') #Aptamer
    B = Strand(capstoneData[i,1], material=nucleicAcid, name='B') #Blocker
    C = Strand(cDNAorRNA(BindingSite+Segment2), material=nucleicAcid, name='C') #CAP 

    # Specify complexes up to size 3
    t1 = Tube(strands={A: concentration, B: concentration, C: concentration},
                complexes=SetSpec(max_size=3), name='t1')
    my_results = tube_analysis(tubes=[t1], model=my_model)
    complex_results = complex_analysis(complexes=t1,model=my_model, compute=['pfunc'])
    complex_results2 = complex_concentrations(tube=t1,data=complex_results)
    [concList[i].append(conc) for i, conc in enumerate(complex_results2.tubes[t1].complex_concentrations.values())]
    totalABCapConc.append(sum(complex_results2.tubes[t1].complex_concentrations.values()))

    if i%250 == 0 and i != 0:
        print(i)

testABC = np.column_stack((capstoneData[:,0], capstoneData[:,1], 
                            a, aa, aaa, aab, aac, ab, abb, abc, ac, acb, acc,
                            b, bb, bbb, bbc, bc, bcc, c, cc, ccc, 
                            totalABCapConc, capstoneIndex))

# Changing matrix to dataframe
labels =['Aptamer (A)','Blocker (B)',
         'A', 'AA', 'AAA', 'AAB', 'AAC', 'AB', 'ABB', 'ABC', 'AC', 'ACB', 'ACC',
         'B', 'BB', 'BBB', 'BBC', 'BC', 'BCC', 'C', 'CC', 'CCC',
         'Total Concentration', 'Index']
labeledCombinedDataABCTesting = pd.DataFrame(testABC, columns=labels)

In [ ]:
# Filtering by Aptamer-CAP concentration
# Main
dataAB = labeledCombinedDataABCTesting.iloc[:,7].astype(float)
dataAC = labeledCombinedDataABCTesting.iloc[:,10].astype(float)
dataTotal = labeledCombinedDataABCTesting.iloc[:,22].astype(float)
abcFilter = (dataAC/dataTotal) > (dataAB/dataTotal)

filteredABCConcData = np.delete(labeledCombinedDataABCTesting.to_numpy(), np.where(~abcFilter), axis=0)
labeledFilteredABCConcData = pd.DataFrame(filteredABCConcData, columns=labeledCombinedDataABCTesting.columns)
print(f"Number of Sequence Pairs: {len(labeledFilteredABCConcData)}")

In [ ]:
# Reframing the concentration graph for the remaining sequences
# Main
totalConc2, concA2, concB2, concAA2, concAB2, concBB2 = [], [], [], [], [], [] # concentration
concList2 = [concA2, concAA2, concAB2, concB2, concBB2]

for i in range(len(labeledFilteredABCConcData)):
    A = Strand(labeledFilteredABCConcData.iloc[i,0], material=nucleicAcid, name='A') #Aptamer
    B = Strand(labeledFilteredABCConcData.iloc[i,1], material=nucleicAcid, name='B') #Blocker
    
    # Specify complexes up to size 3
    t1 = Tube(strands={A: concentration, B: concentration},
              complexes=SetSpec(max_size=2), name='t1')
    my_results = tube_analysis(tubes=[t1], model=my_model)
    
    complex_results = complex_analysis(complexes=t1,model=my_model, compute=['pfunc'])
    complex_results2 = complex_concentrations(tube=t1,data=complex_results)
    [concList2[i].append(conc) for i, conc in enumerate(complex_results2.tubes[t1].complex_concentrations.values())]
    totalConc2.append(sum(complex_results2.tubes[t1].complex_concentrations.values()))

    if i%250 == 0 and i != 0:
        print(i)

testing2 = np.column_stack((labeledFilteredABCConcData.iloc[:,0], labeledFilteredABCConcData.iloc[:,1],
                                concA2, concAA2, concAB2,
                                concB2, concBB2, 
                                totalConc2, labeledFilteredABCConcData.iloc[:,23]))

# Changing matrix to dataframe
labels = ['Aptamer (A)','Blocker (B)',
          'Equilibrium Concentration A', 'Equilibrium Concentration AA', 'Equilibrium Concentration AB', 
          'Equilibrium Concentration B','Equilibrium Concentration BB', 
          'Total Concentration','Index']
labeledCombinedDataTesting2 = pd.DataFrame(testing2, columns=labels) # Converting matrix into dataframe to add labels

In [ ]:
# Graphing the concentrations of the remaining sequences
# Main
concA2, concAA2, concAB2, concB2, concBB2 = [np.array(data) for data in concList2]
capstoneIndex = labeledCombinedDataTesting2.iloc[:,8]
plt.figure(7, figsize=(15,15))
plt.subplot(2,1,1)
barWidth = 0.15
xAxis = np.arange(len(capstoneIndex))
plt.bar(xAxis, (concA2/totalConc2), barWidth, label = "Aptamer (A)")
bottomValue1 = np.array(concA2/totalConc2)
plt.bar(xAxis, concB2/totalConc2, barWidth, label="Blocker (B)", bottom = bottomValue1)
bottomValue1 += np.array(concB2/totalConc2)
plt.bar(xAxis+barWidth, concAA2/totalConc2, barWidth, label="AA")
bottomValue2 = np.array(concAA2/totalConc2)
plt.bar(xAxis+barWidth, concAB2/totalConc2, barWidth, label="AB", bottom = bottomValue2)
bottomValue2 += np.array(concAB2/totalConc2)
plt.bar(xAxis+barWidth, concBB2/totalConc2, barWidth, label="BB", bottom = bottomValue2)
bottomValue2 += np.array(concBB2/totalConc2)
plt.legend(loc='right', bbox_to_anchor=(1.15,0.9))
plt.title('Complex Probabilities')
plt.xticks(xAxis+(1.5)*barWidth, capstoneIndex, rotation = -90)
plt.ylabel('Probabilty')
plt.xlabel('Index')
plt.ylim(0,1)

In [ ]:
# Second Filter for Aptamer-CAP concentrations
# Main
dataAB = labeledFilteredABCConcData.iloc[:,7].astype(float)
dataAC = labeledFilteredABCConcData.iloc[:,10].astype(float)
dataB = labeledFilteredABCConcData.iloc[:,13].astype(float)
dataTotal = labeledFilteredABCConcData.iloc[:,22].astype(float)
abcFilter2 = (dataAC+dataB)/dataTotal > 0.2

# Filtering ABC array
secondFilteredABCConcData = np.delete(labeledFilteredABCConcData.to_numpy(), np.where(~abcFilter2), axis=0)
labeledSecondFilteredABCConcData = pd.DataFrame(secondFilteredABCConcData, columns=labeledFilteredABCConcData.columns)

# Filtering AB array
secondFilteredTesting2 = np.delete(labeledCombinedDataTesting2.to_numpy(), np.where(~abcFilter2), axis=0)
labeledSecondFilteredTesting2 = pd.DataFrame(secondFilteredTesting2, columns=labeledCombinedDataTesting2.columns)
if len(labeledSecondFilteredABCConcData) == len(labeledSecondFilteredTesting2):
    print("AB and ABC Pairs Match!")
    print(f"Number of Sequence Pairs: {len(labeledSecondFilteredABCConcData)}")

In [ ]:
# Filtering and Graphing Final Sequences
# Main
abColumns = list(range(2,7))
abcColumns = list(range(2, 22))

finalABConc = [labeledSecondFilteredTesting2.iloc[:,i].astype(float) for i in abColumns]
concA3, concAA3, concAB3, concB3, concBB3 = [np.array(data) for data in finalABConc]
totalAB = np.sum(finalABConc, axis=0)
capstoneIndex = labeledSecondFilteredTesting2.iloc[:,8]

finalABCConc = [labeledSecondFilteredABCConcData.iloc[:,i].astype(float) for i in abcColumns]
finalIndex = labeledSecondFilteredABCConcData.iloc[:,23]

# Graphing AB
plt.figure(7, figsize=(15,15))
plt.subplot(2,1,1)
barWidth = 0.15
xAxis = np.arange(len(capstoneIndex))
plt.bar(xAxis, (concA3/totalAB), barWidth, label = "Aptamer (A)")
bottomValue1 = np.array(concA3/totalAB)
plt.bar(xAxis, concB3/totalAB, barWidth, label="Blocker (B)", bottom = bottomValue1)
bottomValue1 += np.array(concB3/totalAB)
plt.bar(xAxis+barWidth, concAA3/totalAB, barWidth, label="AA")
bottomValue2 = np.array(concAA3/totalAB)
plt.bar(xAxis+barWidth, concAB3/totalAB, barWidth, label="AB", bottom = bottomValue2)
bottomValue2 += np.array(concAB3/totalAB)
plt.bar(xAxis+barWidth, concBB3/totalAB, barWidth, label="BB", bottom = bottomValue2)
bottomValue2 += np.array(concBB3/totalAB)
plt.legend(loc='right', bbox_to_anchor=(1.15,0.9))
plt.title('Complex Probabilities')
plt.xticks(xAxis+(1.5)*barWidth, capstoneIndex, rotation = -90)
plt.ylabel('Probabilty')
plt.xlabel('Index')
plt.ylim(0,1)

# Graphing ABC
graphingLabels = ['Aptamer (A)', 'AA', 'AAA', 'AAB', 'AAC', 'AB', 'ABB', 'ABC', 'AC', 'ACB', 'ACC',
                  'Blocker (B)', 'BB', 'BBB', 'BBC', 'BC', 'BCC',
                  'CAP (C)', 'CC', 'CCC',]
finalTotals = graphingAptamerBlockerCAP(finalABCConc, finalIndex, "Final Complex Probabilities", graphingLabels)
labeledSecondFilteredABCConcData.to_excel(saveData_PostFilters_Name) 

In [ ]:
# Graphing the concentrations of the forward primer, reverse primer, and blocker sequence 

# THIS IS ONLY FOR THERMODYNAMIC AMPLIFICATION (not enzamatic therefore not applicable to RPA)

# Main
finalAptamers = labeledSecondFilteredABCConcData.iloc[:,0]
blockerRPA = labeledSecondFilteredABCConcData.iloc[:,1]
indexRPA = labeledSecondFilteredABCConcData.iloc[:,23]

concList1To1 = rpaConcentrations(5e-7, 5e-7, blockerRPA)
concList2To1 = rpaConcentrations(1e-6, 5e-7, blockerRPA)
concList1To2 = rpaConcentrations(5e-7, 1e-6, blockerRPA)

graphingRPALabels = ['Foward Primer (F)', 'FF', 'FFF', 'FFB', 'FFR', 'FB', 'FBB', 'FBR', 'FR', 'FRB', 'FRR',
                  'Blocker (B)', 'BB', 'BBB', 'BBR', 'BR', 'BRR',
                  'Reverse Primer (R)', 'RR', 'RRR',]
finalTotals = graphingConcentration3Strand(concList1To1, indexRPA, "1:1 Ratio Complex Probability with Blocker and Primer", graphingRPALabels)
finalTotals = graphingConcentration3Strand(concList2To1, indexRPA, "2:1 Ratio Complex Probability with Blocker and Primer", graphingRPALabels)
finalTotals = graphingConcentration3Strand(concList1To2, indexRPA, "1:2 Ratio Complex Probability with Blocker and Primer", graphingRPALabels)

In [ ]:
# Printing the blockers and aptamer sequences
print("Blockers")
for data in blockerRPA:
    print(data)

print("Aptamers")
for data in finalAptamers:
    print(data)

print(f"Gap: {gap[0]}")